In [12]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix
import scipy.sparse
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

In [13]:
customers = pd.read_csv('D:/H&M/data/customers.csv')
articles = pd.read_csv('D:/H&M/data/articles.csv')
train = pd.read_csv('D:/H&M/data/transactions_train.csv')

In [14]:
ALL_USERS = customers['customer_id'].unique().tolist()
ALL_ITEMS = articles['article_id'].unique().tolist()

encode_User = LabelEncoder().fit(ALL_USERS)
train['user_id'] = encode_User.transform(train['customer_id'])
encode_Item = LabelEncoder().fit(ALL_ITEMS)
train['item_id'] = encode_Item.transform(train['article_id'])
customers['encode_customer_id'] = encode_User.transform(customers['customer_id'])
articles['encode_article_id'] = encode_Item.transform(articles['article_id'])

In [15]:
encode_articles = articles[['article_id','encode_article_id']]

In [16]:
encode_articles.head()

,article_id,encode_article_id
0,108775015,0
1,108775044,1
2,108775051,2
3,110065001,3
4,110065002,4


In [19]:
encode_customers = customers[['customer_id','encode_customer_id']]

In [22]:
encode_articles.to_parquet('encode_articles.parquet')
encode_customers.to_parquet('encode_customers.parquet')

In [23]:
col = train['item_id'].values
row = train['user_id'].values
one = np.ones(train.shape[0])

coo = coo_matrix((one, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
csr = coo.tocsr()

In [24]:
print(coo.shape)

(1371980, 105542)


In [25]:
scipy.sparse.save_npz('user_sparse_matrix.npz', csr)

In [26]:
csr.shape

(1371980, 105542)

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
 5   user_id           int32  
 6   item_id           int64  
dtypes: float64(1), int32(1), int64(3), object(2)
memory usage: 1.5+ GB


In [29]:
train['encode_customer_id'] = encode_User.transform(train['customer_id'])
train['encode_article_id'] = encode_Item.transform(train['article_id'])

In [37]:
each_customer_bought=train[['encode_customer_id','encode_article_id']]

In [42]:
def combine_values(series):
    return ', '.join(map(str, series))

each_customer_boughtGB = each_customer_bought.groupby('encode_customer_id')

In [44]:
each_customer_boughtGB = each_customer_boughtGB['encode_article_id'].agg(combine_values).reset_index()

In [45]:
each_customer_boughtGB.head(10)

,encode_customer_id,encode_article_id
0,0,"29516, 99, 30327, 50724, 16003, 16003, 23996, ..."
1,1,"19333, 33748, 33991, 8216, 41024, 19333, 42626..."
2,2,"40179, 10520, 40179, 18197, 59458, 1469, 1469,..."
3,3,"64525, 61175"
4,4,"32247, 43442, 51124, 54461, 2181, 2181, 20517,..."
5,5,"62961, 44506, 64274"
6,6,"62288, 60135, 57068, 75961, 58295, 3091"
7,7,"6387, 46304, 46305, 46302, 6386, 6700, 8427, 2..."
8,8,"85392, 91655"
9,9,"42625, 12016, 35802, 42628"


In [46]:
each_customer_boughtGB.to_parquet('each_customer_bough.parquet')